In [ ]:
!pip install hdbscan seaborn

In [ ]:
import pandas as pd
import seaborn as sns
from sentence_transformers import SentenceTransformer

https://developers.generativeai.google/examples/clustering_with_embeddings

In [ ]:
device = "mps"

model = SentenceTransformer('all-MiniLM-L6-v2', device=device)

In [ ]:
df_data = pd.read_json("../../data/sft_train_shuffled_reduced.jsonl", lines=True)

In [ ]:
df_data["combined_instruction"] = (df_data["instruction"] + ' ' + df_data["input"]).str.rstrip()

In [ ]:
concatenated_instructions = df_data["combined_instruction"].tolist()

In [ ]:
instruction_embeddings = model.encode(concatenated_instructions)

In [ ]:
N = instruction_embeddings.shape[0]

In [ ]:
from sklearn.manifold import TSNE

tsne = TSNE(random_state=98424, n_iter=10000, metric="cosine", perplexity=int(N ** (1/2)))

In [ ]:
embeddings_tsne = tsne.fit_transform(instruction_embeddings)

In [ ]:
from sklearn.cluster import AgglomerativeClustering

model = AgglomerativeClustering(n_clusters=4, metric="cosine")

labels = model.fit_predict(instruction_embeddings)

In [ ]:
labels.max()

In [ ]:
# from sklearn.cluster import KMeans
# from sklearn.metrics import silhouette_score

# sil_score_max = -1 #this is the minimum possible score

# for n_clusters in range(2,10):
#   model = KMeans(n_clusters = n_clusters, init='k-means++', max_iter=100, n_init=1)
#   labels = model.fit_predict(instruction_embeddings)
#   sil_score = silhouette_score(instruction_embeddings, labels)
#   print("The average silhouette score for %i clusters is %0.2f" %(n_clusters,sil_score))
#   if sil_score > sil_score_max:
#     sil_score_max = sil_score
#     best_n_clusters = n_clusters

In [ ]:
from sklearn.cluster import KMeans

kmeans_model = KMeans(n_clusters=4, random_state=1, n_init='auto').fit(embeddings_tsne)
labels = kmeans_model.fit_predict(instruction_embeddings)

In [ ]:
df_tsne = pd.DataFrame(embeddings_tsne, columns=['x', 'y'])
df_tsne["label"] = labels

In [ ]:
df_tsne["label_int"] = pd.factorize(df_data["dataset"])[0]

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

FS = (20, 16)
fig, ax = plt.subplots(figsize=FS)
sns.scatterplot(data=df_tsne, x='x', y='y', hue='label', palette='tab10', alpha=0.2)
sns.move_legend(ax, "upper left", bbox_to_anchor=(1, 1))
ax.text(55, 25, "Logical reasoning", fontsize=20, fontweight='bold')
ax.text(-130, -25, "Math (numerical reasoning)", fontsize=20, fontweight='bold')
ax.text(-30, 0, "General knowledge", fontsize=20, fontweight='bold')
ax.text(10, 60, "Science", fontsize=20, fontweight='bold')
plt.axis('off')
plt.show()

In [ ]:
df_data["label"] = labels

df_data.assign(x= embeddings_tsne[:,0], y = embeddings_tsne[:,1]).loc[lambda df: df["label"] == 3].value_counts("dataset")#.loc[lambda df: df["y"] < -120]#.loc[lambda df: df["x"] > 40].head(20)

In [ ]:
embeddings_tsne_df = pd.DataFrame(embeddings_tsne, columns=["x", "y"])

idx = embeddings_tsne_df.loc[lambda df: (df["x"] > -10) & (df["x"] < 10) & (df["y"] < 65)].index

idx = embeddings_tsne_df.loc[lambda df: (df["x"] > 52) & (df["y"] < 37.5)].index

In [ ]:
df_data.loc[idx]